In [1]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import Blip2Processor, Blip2ForConditionalGeneration, Blip2Config
from PIL import Image
import requests

In [2]:
CHECK_POINT = "Salesforce/blip2-opt-2.7b"

In [3]:
config = Blip2Config.from_pretrained(CHECK_POINT)
with init_empty_weights():
    model = Blip2ForConditionalGeneration(config)
    model.tie_weights()

/home/yirguo/data/conda_env/cross_modal_homework/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
device_map = infer_auto_device_map(model, no_split_module_classes=["OPTDecoderLayer"], dtype="float32", max_memory={0: "8GiB", 1: "10GiB", 2: "10Gib", 3: "10Gib", 4: "9Gib", 5: "10Gib", 6: "10Gib", 7: "10Gib"})
device_map

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


OrderedDict([('query_tokens', 0),
             ('vision_model', 0),
             ('qformer', 0),
             ('language_projection', 0),
             ('language_model.model.decoder.embed_tokens', 0),
             ('language_model.model.decoder.embed_positions', 0),
             ('language_model.model.decoder.final_layer_norm', 0),
             ('language_model.model.decoder.layers.0', 0),
             ('language_model.model.decoder.layers.1', 0),
             ('language_model.model.decoder.layers.2', 0),
             ('language_model.model.decoder.layers.3', 0),
             ('language_model.model.decoder.layers.4', 0),
             ('language_model.model.decoder.layers.5', 0),
             ('language_model.model.decoder.layers.6', 0),
             ('language_model.model.decoder.layers.7', 0),
             ('language_model.model.decoder.layers.8', 0),
             ('language_model.model.decoder.layers.9', 0),
             ('language_model.model.decoder.layers.10', 1),
             ('l

In [5]:
device_map['language_model.lm_head'] = device_map['language_model.model.decoder.embed_tokens']

In [6]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", device_map=device_map)

img_url = 'https://gker-love.oss-cn-beijing.aliyuncs.com/Naive/messages/6e6c01ed-29bb-447d-8790-4f068d0b6e8a/da6a1872-5d75-478d-a5ac-8e5e24864df4.jpeg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

question = ""
inputs = processor(raw_image, question, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True).strip())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


a large building with a clock tower


: 